In [1]:
import math
import torch

from botorch.fit import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood

X = torch.rand(20, 2) - 0.5
Y = (torch.sin(2 * math.pi * X[:, 0]) + torch.cos(2 * math.pi * X[:, 1])).unsqueeze(-1)
Y += 0.1 * torch.randn_like(Y)

gp = SingleTaskGP(X, Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll);

In [4]:
from botorch.acquisition import UpperConfidenceBound

UCB = UpperConfidenceBound(gp, beta=0.1)

In [8]:
X.unsqueeze(-2).shape

torch.Size([50, 1, 2])

In [19]:
help(cma.CMAOptions)

Help on class CMAOptions in module cma.evolution_strategy:

class CMAOptions(builtins.dict)
 |  CMAOptions(s=None, **kwargs)
 |  
 |  a dictionary with the available options and their default values
 |  for class `CMAEvolutionStrategy`.
 |  
 |  ``CMAOptions()`` returns a `dict` with all available options and their
 |  default values with a comment string.
 |  
 |  ``CMAOptions('verb')`` returns a subset of recognized options that
 |  contain 'verb' in there keyword name or (default) value or
 |  description.
 |  
 |  ``CMAOptions(opts)`` returns the subset of recognized options in
 |  ``dict(opts)``.
 |  
 |  Option values can be "written" in a string and, when passed to `fmin`
 |  or `CMAEvolutionStrategy`, are evaluated using "N" and "popsize" as
 |  known values for dimension and population size (sample size, number
 |  of new solutions per iteration). All default option values are given
 |  as such a string.
 |  
 |  Details
 |  -------
 |  `CMAOptions` entries starting with ``tol

In [18]:
help(cma.CMAEvolutionStrategy)

Help on class CMAEvolutionStrategy in module cma.evolution_strategy:

class CMAEvolutionStrategy(cma.interfaces.OOOptimizer)
 |  CMAEvolutionStrategy(x0, sigma0, inopts=None)
 |  
 |  CMA-ES stochastic optimizer class with ask-and-tell interface.
 |  
 |  Calling Sequences
 |  =================
 |  
 |  - ``es = CMAEvolutionStrategy(x0, sigma0)``
 |  
 |  - ``es = CMAEvolutionStrategy(x0, sigma0, opts)``
 |  
 |  - ``es = CMAEvolutionStrategy(x0, sigma0).optimize(objective_fct)``
 |  
 |  - ::
 |  
 |      res = CMAEvolutionStrategy(x0, sigma0,
 |                              opts).optimize(objective_fct).result
 |  
 |  Arguments
 |  =========
 |  `x0`
 |      initial solution, starting point. `x0` is given as "phenotype"
 |      which means, if::
 |  
 |          opts = {'transformation': [transform, inverse]}
 |  
 |      is given and ``inverse is None``, the initial mean is not
 |      consistent with `x0` in that ``transform(mean)`` does not
 |      equal to `x0` unless ``transfor

In [46]:
encoding_length = 2
bounds = torch.tensor([[-1.0] * encoding_length, [1.0] * encoding_length])
bounds.tolist()

np.arctanh(np.random.normal(loc=0.5,scale=0.4,size=encoding_length))

/Users/trongduong/opt/anaconda3/envs/QML/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in arctanh
  """


array([       nan, 0.34701545])

In [60]:
cma.CMAOptions.defaults()

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA',
 'CMA_active': 'True  # negative update, conducted after the original update',
 'CMA_active_injected': '0  #v weight multiplier for negative weights of injected solutions',
 'CMA_cmean': '1  # learning rate for the mean value',
 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid',
 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always',
 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors',
 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance',
 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold',
 'CMA_mirrors': 'popsize < 6  # values <0.5 are interpreted as fraction, values >1 as numbers (rounded), for `True` about 0.16 is used',
 'CM

In [63]:
import cma
import numpy as np

# get initial condition for CMAES in numpy form
# note that CMAES expects a different shape (no explicit q-batch dimension)
x0 = np.random.normal(loc=0.5,scale=0.4,size=2)

# create the CMA-ES optimizer
es = cma.CMAEvolutionStrategy(
    x0=x0,
    sigma0=0.2,
    #inopts={"popsize": 128, "bounds": bounds.tolist()},
    inopts={"popsize": 50, "maxiter":100}
)

# speed up things by telling pytorch not to generate a compute graph in the background
with torch.no_grad():

    # Run the optimization loop using the ask/tell interface -- this uses 
    # PyCMA's default settings, see the PyCMA documentation for how to modify these
    while not es.stop():
        xs = es.ask()  # as for new points to evaluate
        # convert to Tensor for evaluating the acquisition function
        X = torch.tensor(xs, device=X.device, dtype=X.dtype)
        # evaluate the acquisition function (optimizer assumes we're minimizing)
        Y = - UCB(X.unsqueeze(-2))  # acquisition functions require an explicit q-batch dimension
        y = Y.view(-1).double().numpy()  # convert result to numpy array
        print(X.shape, Y.shape, y.shape)
        #print(y)
        es.tell(xs, y)  # return the result to the optimizer

# convert result back to a torch tensor
best_x = torch.from_numpy(es.best.x).to(X)

print(best_x)
print(es.best.f)

(25_w,50)-aCMA-ES (mu_w=14.0,w_1=14%) in dimension 2 (seed=334130, Thu Jan 27 23:51:23 2022)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torch.Size([50, 2]) torch.Size([50]) (50,)
torc

In [22]:
es.result

CMAEvolutionStrategyResult(xbest=array([0.23287301, 0.01928219]), fbest=-1.995552659034729, evals_best=1427, evaluations=11000, iterations=220, xfavorite=array([0.23278461, 0.0193827 ]), stds=array([5.28270410e-10, 7.90330536e-10]), stop={'tolfunhist': 1e-12})

In [13]:
es.best.f

-1.9955530166625977

In [32]:
candidates = torch.tensor([]*4)
x = torch.tensor([1,2,3,4])
torch.stack((candidates,x),dim=0)

RuntimeError: stack expects each tensor to be equal size, but got [0] at entry 0 and [4] at entry 1

In [51]:
num_ops = 10
np.linspace(0,10, 2*num_ops+1)[1::2]

array([0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5])

In [53]:
d = 5

bounds = torch.stack([-torch.ones(d), torch.ones(d)])

train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(50, d)

In [67]:
np.array(list(zip(bounds[0], bounds[1])))

array([[-1.,  1.],
       [-1.,  1.],
       [-1.,  1.],
       [-1.,  1.],
       [-1.,  1.]], dtype=float32)

In [70]:
a = np.array([1,2,3,4,5])
x = torch.from_numpy(a).unsqueeze(-1)
x

tensor([[1],
        [2],
        [3],
        [4],
        [5]])